# **Preprocessing avant utilisation des modèles**

Ce notebook vise à explorer rapidement la base de données pour avoir une idée de se composition, la répartition et éventuels déséquilibres entre les classes, les caractéristiques des textes à analyser. Cela est éventuellement l'occasion de détecter des erreurs de textes (encodage erroné au moment d'enregistrer/lire les demandes)

In [7]:
# Pour faciliter la mise à jour des fonctions écrites dans func_custom sans avoir à redémarrer le kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Packages classiques
import pandas as pd
import numpy as np

# Custom package
import func_custom as fc

# NLP
import unidecode
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import spacy

In [9]:
df = pd.read_excel("data/data_train.xlsx",
                    usecols = ["label", "message"])

# 1. Analyse, justification et étapes du preprocessing

Si certaines étapes sont standards et ne nécessite pas d'explication (mettre en .lower() par exemple), regardons certains points plus particulièrement

## 1.1 Application de .lower() : non commenté

## 1.2 Encodage des accents

In [10]:
first_message = df["message"][0].split()
data_encodage = [first_message[i] for i in [21, 37, 43, 46]]
print(data_encodage)
keyboard_encodage = ["prélèvement", "époux", "à", "impôts"]
print(keyboard_encodage)

['prélèvement', 'époux', 'à', 'impôts']
['prélèvement', 'époux', 'à', 'impôts']


In [11]:
print("Encodage des accents")
print([t.encode("unicode_escape") for t in data_encodage])
print([t.encode("unicode_escape") for t in keyboard_encodage])

print("Conséquence sur la lemmatization")
stemmer = SnowballStemmer('french')
print([stemmer.stem(t) for t in data_encodage])
print([stemmer.stem(t) for t in keyboard_encodage])

print("Conséquence sur l'embedding")
nlp = spacy.load("fr_core_news_md")
print([np.sum(nlp(t)[0].vector) for t in data_encodage])
print([np.sum(nlp(t)[0].vector) for t in keyboard_encodage])

print("Encodage après normalisation")
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in data_encodage])
print([unicodedata.normalize("NFKC", t).encode("unicode_escape") for t in keyboard_encodage])

print("Embedding après normalisation")
print([np.sum(nlp(unicodedata.normalize("NFKC", t))[0].vector) for t in data_encodage])
print([np.sum(nlp(unicodedata.normalize("NFKC", t))[0].vector) for t in keyboard_encodage])

Encodage des accents
[b'pre\\u0301le\\u0300vement', b'e\\u0301poux', b'a\\u0300', b'impo\\u0302ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']
Conséquence sur la lemmatization
['prélèv', 'époux', 'à', 'impôt']
['prélev', 'époux', 'à', 'impôt']
Conséquence sur l'embedding
[0.0, 0.0, -37.616055, 0.0]
[-0.74289227, 20.48502, -41.42349, 39.59558]
Encodage après normalisation
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']
[b'pr\\xe9l\\xe8vement', b'\\xe9poux', b'\\xe0', b'imp\\xf4ts']
Embedding après normalisation
[-0.74289227, 20.48502, -41.42349, 39.59558]
[-0.74289227, 20.48502, -41.42349, 39.59558]


Il faudra une étape de normalisation avec `unicodedata.normalize("NFKC", text)` sinon l'embedding de Spacy utilisé par la suite retournement des vecteurs nuls.

## 1.3 Gestion de la ponctuation

Les signes de ponctuations sont tous remplacés par un _espace_ pour anticiper sur les failles du tokenizer de nltk. Typiquement en cas d'erreur de phrase, d'absence d'espace, surtout autour de mots inconnus le tokenizer rate complètement. Exemple avec cette phrase qui contient `0000.L'XXXXX́e` :

In [12]:
message_test = df[df["message"].str.contains("0000.L'XXXXX́e")]["message"].values[0]
print(message_test)

Bonjour, J'ai divorcé en 0000.J'ai commencé à verser une pension alimentaire à partir de septembre 0000.L'XXXXX́e dernière j'ai donc versé un XXXXX́e pleine.Par rapport au prélèvement à la source comment cela ce passe-t-il sachant que j'aurais plus à déduire pour l'XXXXX́e dernière que 0000?actuellement je suis prélever par rapport à 0000?y-aura t-il un remboursement en fin d'XXXXX́e?Peut-on modifier le prélèvement à la source en cours d'XXXXX́e? D'avance merci Cordialement XXXXX XXXXX


In [13]:
tokens = word_tokenize(message_test, language = "french")

On constate un problème avec le tokenizer qui persisterait si on supprimait simplement la ponctuation, je préfère donc la remplacer par un espace.

In [14]:
tokens[10:20]

['pension',
 'alimentaire',
 'à',
 'partir',
 'de',
 'septembre',
 "0000.L'XXXXX́e",
 'dernière',
 "j'ai",
 'donc']

In [15]:
tokens = word_tokenize(fc.replace_punctuation_with_space(message_test), language = "french")

In [16]:
tokens[15:25]

['partir',
 'de',
 'septembre',
 '0000',
 'L',
 'XXXXX́e',
 'dernière',
 'j',
 'ai',
 'donc']

## 1.4 Tokenizer

J'utilise ici le tokenizer par défaut de nltk en français

## 1.5 Suppression des caractères de taille 1

Pour prendre en compte des caractères spéciaux ou des lettres uniques suite au tokenizer, qui n'apporteront pas d'informations. Cela permet de limiter la taille des stopwords à inclure. Par ailleurs avec l'encodage diacritique la longueur de `à` serait en réalité de deux.

## 1.6 Gestion des stopwords

En combinant tous les éléments précédents dans la fonction `preprocess_stopwords` dans `func_custom.py` étudions désormais le traitement des stopwords. 
Commençons pas ne prendre en compte que ceux de base dans nltk :

In [17]:
stopwords_french = set(stopwords.words('french'))

In [18]:
text = ' '.join(df["message"].dropna())
tokens_clean = fc.preprocess_stopwords(text, stopwords_french)
pd.DataFrame(tokens_clean).value_counts().head(15)

0           
xxxxx           1777
taux            1323
revenus          712
prélèvement      670
bonjour          624
source           526
cordialement     413
plus             366
merci            365
déclaration      300
comment          271
faire            260
bien             256
situation        255
xxxxx́e          253
Name: count, dtype: int64

On constate que la simple inclusion des stopwords par défaut de nltk ne suffira pas, des éléments spécifiques au jeu de données (`xxxxx` et `0000` issus de pseudonymisation, `bonjour` et `merci` et mots de politesse du fait qu'il s'agit de message écrits par des particuliers à la DGFIP). On peut alors compléter de manière _ad hoc_ cette liste en regardant à l'oeil nu ces éléments. Si l'exerice consiste cependant à distinguer les messages "polis" des autres alors cette liste de stopwords n'est absolument pas pertinente.

TF-IDF aurait dans une certaine mesure pu tenir compte de ces mots largement présent dans le corpus et peu informatif, autant traiter ce problème à la racine ce qui limitera la taille des données à traiter par la suite.

In [19]:
stopwords_adhoc = {"à", "xxxxx", "bonjour", "cordialement", "merci", "xxxxx́e", "xxxxx́", "k€", "donc", "car", "cette", "cela",
                  "être", "si", "même", "faire", "avoir", "remercie", "madame", "monsieur"}
stopwords_complete = stopwords_french.union(stopwords_adhoc)

In [20]:
tokens_clean = fc.preprocess_stopwords(text, stopwords_complete)
pd.DataFrame(tokens_clean).value_counts().head(60)

0            
taux             1323
revenus           712
prélèvement       670
source            526
plus              366
déclaration       300
comment           271
bien              256
situation         255
mois              249
retraite          232
depuis            229
salaire           213
impôt             210
compte            201
avance            201
euros             192
individualisé     183
réponse           181
impôts            180
épouse            176
revenu            174
janvier           174
montant           166
pension           163
fait              163
conjoint          154
dois              151
imposition        145
changement        145
suite             141
espace            134
pouvez            128
part              126
emploi            126
deux              112
prélevé           111
foyer             110
alors             110
déclarer          107
payer             105
mari              104
non               100
pacs               99
concernant        

## 1.7 Prise en compte des fautes d'orthographe : une méthode simple

Je ne vais pas essayer de prendre en compte exhaustivement des fautes d'orthographe mais me contenter d'identifier les erreurs les plus fréquentes dans le corpus. Cela me permettra 1) de corriger les erreurs fréquentes mais utile à garder (par exemple un accent manquant dans prélèvement) et 2) augmenter retrospectivement la liste des stopwords avec des abbréviations non pertinentes (`mme`, `mr`, `svp`).

**Important , on constate la présence récurrente du PACS or on peut anticiper qu'il y aura un soucis de vocabulaire lors des méthodes d'embedding.**

In [21]:
with open("data/mots dictionnaires.txt", "r", encoding= "utf-8") as fichier:
   # Lire toutes les lignes du fichier et les stocker dans une liste
   dic_french = set([(unicodedata.normalize("NFKC", ligne.strip().lower())) for ligne in fichier.readlines()])

In [22]:
typo_investiguer = []
for elem in tokens_clean:
    if not elem in dic_french:
        typo_investiguer.append(elem)

In [23]:
pd.DataFrame(typo_investiguer).value_counts().head(50)

0          
pacs           99
mme            87
er             62
impots         55
pacsé          53
xx             51
mr             44
pole           39
pacsée         28
svp            28
impot          27
pacsés         26
aujourd        26
gouv           24
bnc            21
n°             19
cdd            18
prelevement    17
prélévement    17
france         17
carsat         16
aout           16
connaitre      14
cdi            14
etant          14
fr             13
agirc          12
cdt            11
eur            11
xxxxx́s        11
plait          10
rib            10
prélevement    10
xxxxx́es        9
ir              9
ca              9
meme            8
xxxxxe          8
arrco           8
email           8
chomage         8
pls             7
puisqu          7
fip             7
mlle            6
reponse         6
etre            6
infos           6
csg             6
tns             6
Name: count, dtype: int64

In [24]:
stopwords_adhoc = {"à", "xxxxx", "bonjour", "cordialement", "merci", "xxxxx́e", "xxxxx́", "k€", "donc", "car", "cette", "cela",
                  "être", "si", "même", "faire", "avoir", "remercie", "madame", "monsieur",
                  "mme", "mr", "er", "xx", "svp"}
stopwords_complete = stopwords_french.union(stopwords_adhoc)

In [25]:
correction_list = {
    "impot" : "impôt",
    "prélévement" : "prélèvement",
    "prelevement" : "prélèvement",
    "pole" : "pôle"
}

## 1.8 Stemmer

In [26]:
stemmer = SnowballStemmer('french')
# tokens = [stemmer.stem(mot) for mot in tokens]

In [27]:
stemmer.stem("prélèvement")

'prélev'

In [28]:
stemmer.stem("prélevé")

'prélev'

# 2. Agrégation dans une fonction unique et application

In [29]:
df["message_clean"] = df["message"].apply(lambda x : fc.preprocess_text(x, stopwords_complete))

In [30]:
message_test = df.sample(1)
print(message_test["message"].values[0])
print(message_test["message_clean"].values[0])

Bonjour, Je me permets de vous contacter afin d'obtenir une réponse sur ma déclaration de revenus. Nous avons effectué notre déclaration commune avec ma conjointe (nous ne l'avons pas encore validée), et on nous informe un taux pour le prélèvement à la source pour le foyer de 0000,0000% ou bien, si on le souhaite, un taux individualisé de 0000,0000% pour ma conjointe et de 0000,0000% pour le mien. Actuellement je disposais d'un taux individuel de 0000,0000% comment cela se fait t-il que mon taux individuel augmente de 0000,0000% alors que je n'ai pas déclaré plus que l'XXXXX́e passée ? Ci-dessous extrait du résumé de déclaration : "Vous venez de signaler un changement de situation de famille au titre de l'XXXXX́e 0000. Cet événement impacte votre situation au regard du prélèvement à la source. En conséquence, le taux de prélèvement à la source qui sera transmis aux organismes vous versant des revenus est de : 0000,0000 % . Il est applicable immédiatement. Aprè

In [31]:
df.to_pickle("data/data_clean.pkl")

In [32]:
df.to_csv("data/data_clean.csv", 
                sep = ";",
                index = False)